<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,5896567601480310696,342825
1,9068386407968572094,330863
2,260598435387740869,320332
3,2700219941840496344,307395
4,454960867574323744,346515


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13260 examples in the train dataset and 3333 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,5.368256,False,12,1,39.0,3765901273411398433
1,7.438397,True,13,1,40.0,2962886928991417684
2,7.068020,True,13,1,29.0,2962886928991417684
3,6.878423,False,13,1,42.0,5107972924983092617
4,7.374463,False,13,1,40.0,6749419160304400104


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13252.000000,13260.000000,13260.000000,13173.000000,1.326000e+04
mean,7.240457,27.442157,1.034766,38.628483,4.367715e+18
std,1.302591,6.129277,0.195155,2.523948,2.794161e+18
min,0.518086,12.000000,1.000000,17.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,1.000000,39.000000,3.765901e+18
75%,8.035849,32.000000,1.000000,40.000000,7.170970e+18
max,13.249782,50.000000,4.000000,47.000000,9.183606e+18


In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,5.368256,False,12,Single(1),39.0,3765901273411398433
1,7.438397,True,13,Single(1),40.0,2962886928991417684
2,7.068020,True,13,Single(1),29.0,2962886928991417684
3,6.878423,False,13,Single(1),42.0,5107972924983092617
4,7.374463,False,13,Single(1),40.0,6749419160304400104


In [11]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13255,6.426475,Unknown,47,Single(1),37.0,1002950341933487066
13256,7.231162,Unknown,48,Single(1),41.0,9068386407968572094
13257,8.223242,Unknown,48,Single(1),38.0,6888635180361964765
13258,6.069326,Unknown,50,Single(1),38.0,2768780346491825728
13259,2.874828,Unknown,50,Single(1),32.0,6637442812569910270


In [12]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26334.000000,26334.000000,26334.000000,2.633400e+04
mean,7.241135,27.448242,38.631275,4.367869e+18
std,1.301201,6.129394,2.515147,2.793187e+18
min,0.518086,12.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.622638e+18
50%,7.312733,27.000000,39.000000,3.765901e+18
75%,8.035849,32.000000,40.000000,7.170970e+18
max,13.249782,50.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [14]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6610 eval.csv
  26334 train.csv
  32944 total
==> eval.csv <==
4.81269117946,True,13,Single(1),34.0,4740473290291881219
6.0009827716399995,True,13,Single(1),40.0,6392072535155213407
6.62930021834,True,14,Single(1),38.0,7517141034410775575
6.63150484096,False,14,Single(1),38.0,7517141034410775575
7.68751907594,True,14,Single(1),36.0,7146494315947640619
6.43088418254,True,14,Single(1),38.0,6365946696709051755
7.41194124844,True,14,Single(1),34.0,6141045177192779423
7.5618555866,True,15,Single(1),41.0,7517141034410775575
6.2501051276999995,True,15,Single(1),36.0,7517141034410775575
7.87491199864,False,15,Single(1),39.0,1088037545023002395

==> train.csv <==
5.3682560797,False,12,Single(1),39.0,3765901273411398433
7.43839671988,True,13,Single(1),40.0,2962886928991417684
7.06802011972,True,13,Single(1),29.0,2962886928991417684
6.8784225744,False,13,Single(1),42.0,5107972924983092617
7.3744626639,False,13,Single(1),40.0,6749419160304400104
6.1244416383599996,False,14,Single(1),39.0,742027

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License